In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:

import os
import pyarrow.parquet as pq
import pyarrow as pa
import glob
import time
start_time = time.time()
import datetime

In [4]:
# Import necessary libraries
import pandas as pd
import json
import numpy as np
from tqdm import tqdm
from IPython.display import display

In [5]:
# This script processes a Parquet file containing event data, cleans and transforms it, 
# categorizes events into specific categories (Hotel, Domestic and International Airlines), and creates subsets of the data for further analysis.
# It also formats timestamps and handles missing values in the user_id column.
# The script uses pandas for data manipulation and tqdm for progress tracking.
# The final output includes two DataFrames: one for Hotel events and another for Domestic Airlines and Airlines Payment events.



# Load the Parquet file into a DataFrame
par = pd.read_parquet(r'/Users/daviddangol/Documents/monthly report/domestic airlines/data/query-results_query-with-param_hotel_airlines_intl_bq_param_airlines_only_10-15-2025_11-01-2025000000000000.parquet', engine='pyarrow')

#par = pd.read_parquet(r"C:\Users\lenovo\Downloads\bq data\user_engagement June 26 - July 26\merged_file.parquet", engine='pyarrow')
# Drop rows with missing user_id values
par.dropna(subset=['user_id'], inplace=True)

# Extract numeric part from user_id and handle NaNs
par['user_id'] = par['user_id'].astype(str).str.extract(r'(\d+)').fillna(0).astype(int)

# Convert event_date to datetime format
par['event_date'] = pd.to_datetime(par['event_date'], format='%Y%m%d')

# Convert event_timestamp to a readable datetime format
par['event_timestamp'] = pd.to_datetime(par['event_timestamp'], unit='us')

# Format event_timestamp as yyyy-mm-dd hh:mm:ss
par['event_timestamp'] = par['event_timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Add a new column 'category' based on specific conditions
# Categorize events into 'Airlines Payment', 'Domestic Airlines', 'International Airlines', or 'Hotel' 
conditions = [
    # Combine 'airlines_payment' with 'Domestic Airlines'
    par['event_name'].str.contains('domestic', case=False) | (par['event_name'] == 'airlines_payment'),  # Domestic Airlines condition
    #par['event_name'] == 'airlines_payment', ##Uncomment this line if you want to separate Airlines Payment from Domestic Airlines
    #par['event_name'].str.contains('international', case=False),  # International Airlines condition
    #par['event_name'].str.contains('hotel', case=False)  # Hotel condition
]

choices = [
    #'Airlines Payment',  # Uncomment this line if you want to separate Airlines Payment from Domestic Airlines
    'Domestic Airlines',
    #'International Airlines',
    #'Hotel'
]

# Create the new column 'category' based on the conditions
par['category'] = np.select(conditions, choices, default='Other')

# Group by user_id and aggregate unique category values into a comma-separated string
par['unique_category'] = par.groupby('user_id')['category'].transform(lambda x: ','.join(x.unique()))

# Sort the DataFrame by user_id in ascending order
par = par.sort_values(by='user_id', ascending=True)

# Get the list of columns in the DataFrame
cols = list(par.columns)

# Move 'category' and 'unique_category' columns to appear after 'event_name'
cols.insert(cols.index('event_name') + 1, cols.pop(cols.index('category')))
cols.insert(cols.index('event_name') + 2, cols.pop(cols.index('unique_category')))

# Reorder the DataFrame columns
par = par[cols]

# Create a subset of the data for 'Hotel' category
#Hotel = par[par['category'] == 'Hotel']

# Create a subset of the data for 'Domestic Airlines' and 'Airlines Payment' categories
Domestic_Airlines = par[par['category'].isin(['Domestic Airlines', 'Airlines Payment'])]

# Display the first 5 rows of the 'Hotel' DataFrame
#display(Hotel.head(5))

# Display the first 5 rows of the 'Domestic_Airlines' DataFrame
display(Domestic_Airlines.head(5))

,user_id,event_name,category,unique_category,event_date,event_timestamp,event_params,is_active_user,medium,source,platform,version,install_source
508110,32,domesticFlightConfirmation,Domestic Airlines,Domestic Airlines,2025-10-18,2025-10-18 01:54:37,"[{'key': 'firebase_event_origin', 'value': {'s...",None,(none),(direct),IOS,5.5.27,iTunes
228449,42,domesticFlightDetail,Domestic Airlines,Domestic Airlines,2025-10-25,2025-10-25 13:18:46,"[{'key': 'time', 'value': {'string_value': '19...",None,(none),(direct),ANDROID,4.7.0.1,com.android.vending
146175,42,domesticFlightDetail,Domestic Airlines,Domestic Airlines,2025-10-26,2025-10-26 12:28:47,"[{'key': 'firebase_event_origin', 'value': {'s...",None,(none),(direct),ANDROID,4.7.0.1,com.android.vending
208277,42,domesticFlightSearch,Domestic Airlines,Domestic Airlines,2025-10-25,2025-10-25 14:15:44,"[{'key': 'firebase_event_origin', 'value': {'s...",None,(none),(direct),ANDROID,4.7.0.1,com.android.vending
272686,54,domesticFlightDetail,Domestic Airlines,Domestic Airlines,2025-10-18,2025-10-18 07:13:50,"[{'key': 'time', 'value': {'string_value': '12...",None,organic,google-play,ANDROID,4.7.0.1,com.android.vending


In [6]:

#filter hotel date based on event-date where date is 2025-06-02
# Hotel = Hotel[Hotel['event_date'] == '2025-06-02']
# Display the shape of the 'Hotel' DataFrame
#print("Hotel DataFrame shape:", Hotel.shape)
# Display the shape of the 'Domestic_Airlines' DataFrame
print("Domestic Airlines DataFrame shape:", Domestic_Airlines.shape)

Domestic Airlines DataFrame shape: (542336, 13)


In [7]:
import pandas as pd
import json
import ast
import numpy as np
from tqdm import tqdm
from IPython.display import display



# Function to extract key-value pairs from the 'event_params' column
def extract_key_value_dict(event_params):
    try:
        # Handle numpy array case
        if isinstance(event_params, np.ndarray):
            parsed_params = event_params.tolist()
        # Handle list type directly
        elif isinstance(event_params, list):
            parsed_params = event_params
        # Handle string representations
        elif isinstance(event_params, str):
            try:
                # Attempt JSON parsing first
                parsed_params = json.loads(event_params.replace("}{", "},{"))
            except json.JSONDecodeError:
                # Fallback to Python literal evaluation for single-quoted strings
                try:
                    parsed_params = ast.literal_eval(event_params)
                except (ValueError, SyntaxError):
                    # If all else fails, treat as malformed data
                    return {}
        else:
            return {}

        # Extract nested event_params if present in a dict
        if isinstance(parsed_params, dict) and "event_params" in parsed_params:
            parsed_params = parsed_params["event_params"]
        
        # Convert to list if we have a numpy array after processing
        if isinstance(parsed_params, np.ndarray):
            parsed_params = parsed_params.tolist()
            
        # Ensure we have a list to process
        if not isinstance(parsed_params, list):
            return {}

        key_value_dict = {}
        for param in parsed_params:
            try:
                # Extract key and value from each parameter
                key = param.get('key')
                value_dict = param.get('value', {})
                value = (
                    value_dict.get('string_value') or
                    value_dict.get('int_value') or
                    value_dict.get('float_value') or
                    value_dict.get('double_value')
                )
                if key:
                    key_value_dict[key] = value
            except (AttributeError, TypeError):
                # Skip malformed entries
                continue
                
        return key_value_dict

    except Exception as e:
        # Log any errors encountered during processing
        print(f"Error processing event_params: {str(e)}")
        return {}

# Function to process a DataFrame and extract key-value pairs from the 'event_params' column
def process_dataframe(df):
    key_value_dicts = []
    # Iterate through each row in the 'event_params' column with a progress bar
    for event_params in tqdm(df['event_params'], desc="Processing rows"):
        key_value_dicts.append(extract_key_value_dict(event_params))
    
    # Create a new DataFrame from the extracted key-value pairs
    key_value_df = pd.DataFrame(key_value_dicts)
    # Find the index of the 'event_params' column to insert new columns next to it
    event_params_index = df.columns.get_loc('event_params')
    
    # Insert the new columns into the original DataFrame
    for idx, col in enumerate(key_value_df.columns, start=1):
        df.insert(event_params_index + idx, col, key_value_df[col])


# Hotel = pd.read_parquet(r"D:\Asar12Hotel.parquet", engine='pyarrow')
# Process the Hotel and Domestic_Airlines DataFrames
#process_dataframe(Hotel)
process_dataframe(Domestic_Airlines)

# Display the first few rows of the updated DataFrames
# display(Hotel.head(10))
display(Domestic_Airlines.head())

Processing rows: 100%|██████████| 542336/542336 [00:07<00:00, 74435.37it/s] 


,user_id,event_name,category,unique_category,event_date,event_timestamp,event_params,firebase_event_origin,ga_session_id,firebase_screen_class,...,user_type,logout_from,accountHolderName,esewaAccountId,is_active_user,medium,source,platform,version,install_source
508110,32,domesticFlightConfirmation,Domestic Airlines,Domestic Airlines,2025-10-18,2025-10-18 01:54:37,"[{'key': 'firebase_event_origin', 'value': {'s...",app,1.761721e+09,SearchFlightActivity,...,NaN,NaN,NaN,NaN,None,(none),(direct),IOS,5.5.27,iTunes
228449,42,domesticFlightDetail,Domestic Airlines,Domestic Airlines,2025-10-25,2025-10-25 13:18:46,"[{'key': 'time', 'value': {'string_value': '19...",app,1.761485e+09,SearchFlightActivity,...,NaN,NaN,NaN,NaN,None,(none),(direct),ANDROID,4.7.0.1,com.android.vending
146175,42,domesticFlightDetail,Domestic Airlines,Domestic Airlines,2025-10-26,2025-10-26 12:28:47,"[{'key': 'firebase_event_origin', 'value': {'s...",app,1.761351e+09,NaN,...,NaN,NaN,NaN,NaN,None,(none),(direct),ANDROID,4.7.0.1,com.android.vending
208277,42,domesticFlightSearch,Domestic Airlines,Domestic Airlines,2025-10-25,2025-10-25 14:15:44,"[{'key': 'firebase_event_origin', 'value': {'s...",app,1.761636e+09,NewAirlinesMainViewController,...,NaN,NaN,NaN,NaN,None,(none),(direct),ANDROID,4.7.0.1,com.android.vending
272686,54,domesticFlightDetail,Domestic Airlines,Domestic Airlines,2025-10-18,2025-10-18 07:13:50,"[{'key': 'time', 'value': {'string_value': '12...",app,1.761274e+09,SearchFlightActivity,...,NaN,NaN,NaN,NaN,None,organic,google-play,ANDROID,4.7.0.1,com.android.vending


In [9]:
Domestic_Airlines.to_parquet(r"/Users/daviddangol/Documents/monthly report/domestic airlines/data/domestic_airline_oara", index=False)

In [10]:
# Display the first row of the 'event_params' column from the Hotel DataFrame in case the conversion fails
# print("First row of 'event_params' from Hotel DataFrame:") #uncomment this line to display the first row
# print(Hotel['event_params'].head(1).to_string(index=False, header=False)) #uncomment this line to display the first row
# Hotel['event_params'].head(1).to_clipboard(index=False, header=False) #uncomment this line to copy the first row to clipboard

#Get all columns from Hotel and Domestic Airlines DataFrames and display unique list of columns
# hotel_columns = Hotel.columns.tolist()
domestic_airlines_columns = Domestic_Airlines.columns.tolist()
# all_columns = list(set(hotel_columns + domestic_airlines_columns))

#print("Unique columns in both DataFrames:")
# print(all_columns)

# Display the columns of each DataFrame
# print("Hotel DataFrame columns:")
# print(Hotel.columns)
print("Domestic Airlines DataFrame columns:")
print(Domestic_Airlines.columns)

Domestic Airlines DataFrame columns:
Index(['user_id', 'event_name', 'category', 'unique_category', 'event_date',
       'event_timestamp', 'event_params', 'firebase_event_origin',
       'ga_session_id', 'firebase_screen_class', 'ga_session_number',
       'confirmed', 'departureCity', 'firebase_conversion',
       'departureFlightNumber', 'engaged_session_event',
       'departureFlightClassCode', 'adults', 'firebase_screen_id', 'tripType',
       'children', 'destinationCity', 'departureAircraftType',
       'navigated_from', 'time', 'departureDate', 'flightClassCode',
       'aircraftType', 'flightNumber', 'fallback_api', 'returnAircraftType',
       'returnFlightNumber', 'returnFlightClassCode', 'user_type',
       'logout_from', 'accountHolderName', 'esewaAccountId', 'is_active_user',
       'medium', 'source', 'platform', 'version', 'install_source'],
      dtype='object')


In [11]:
#Function to drop columns from Hotel and Domestic Airlines DataFrames
def drop_columns(df):
    columns_to_drop = [
        'event_params', 'flightClassCode', 'departureFlightNumber', 'logout_from', 'engaged_session_event',
        'returnAircraftType', 'is_active_user', 'departureFlightClassCode', 'firebase_screen_class',
        'listingFrom', 'returnFlightClassCode', 'ga_session_number', 'hotelId', 'flightNumber', 'isFilterApplied',
        'firebase_screen_id', 'fallback_api', 'aircraftType', 'navigated_from', 'firebase_conversion', 'firebase_event_origin',
        'returnFlightNumber', 'ga_session_id'
        
    ]
    # Return new DataFrame without the columns (doesn't modify original)
    return df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')
    return df

#Hotel1 = drop_columns(Hotel)
Domestic_Airlines1 = drop_columns(Domestic_Airlines)
#display(Hotel1.head(50))
display(Domestic_Airlines1.head(5))

,user_id,event_name,category,unique_category,event_date,event_timestamp,confirmed,departureCity,adults,tripType,...,time,departureDate,user_type,accountHolderName,esewaAccountId,medium,source,platform,version,install_source
508110,32,domesticFlightConfirmation,Domestic Airlines,Domestic Airlines,2025-10-18,2025-10-18 01:54:37,NaN,Kathmandu,1,One Way,...,12:41:14,2025-10-31,NaN,NaN,NaN,(none),(direct),IOS,5.5.27,iTunes
228449,42,domesticFlightDetail,Domestic Airlines,Domestic Airlines,2025-10-25,2025-10-25 13:18:46,NaN,Surkhet,1,One Way,...,19:07:57,2025-11-01,NaN,NaN,NaN,(none),(direct),ANDROID,4.7.0.1,com.android.vending
146175,42,domesticFlightDetail,Domestic Airlines,Domestic Airlines,2025-10-26,2025-10-26 12:28:47,NaN,Nepalgunj,1,Two Way,...,01:58:14,2025-10-25,NaN,NaN,NaN,(none),(direct),ANDROID,4.7.0.1,com.android.vending
208277,42,domesticFlightSearch,Domestic Airlines,Domestic Airlines,2025-10-25,2025-10-25 14:15:44,NaN,Pokhara,1,One Way,...,NaN,NaN,NaN,NaN,NaN,(none),(direct),ANDROID,4.7.0.1,com.android.vending
272686,54,domesticFlightDetail,Domestic Airlines,Domestic Airlines,2025-10-18,2025-10-18 07:13:50,NaN,Nepalgunj,1,One Way,...,08:38:35,2025-10-25,NaN,NaN,NaN,organic,google-play,ANDROID,4.7.0.1,com.android.vending


In [12]:
#Display unique value in address column in Hotel1 DataFrame
unique_address = Domestic_Airlines1['destinationCity'].unique()
# print("Unique values in 'address' column:")
# print(unique_address)
#unique_hotel_destination = Hotel1['destination'].unique()
#print("Unique values in 'hotel_destination' column:")
print(unique_address)
# Save the processed DataFrames to CSV files
# Hotel.to_csv(r"D:\Hotel_Experiment\Hotel_exp_0603-0604.csv", index=False)

['Nepalgunj' 'Kathmandu' 'Talcha' 'Dhangadhi' 'KATHMANDU' 'Bhadrapur'
 'Pokhara' 'Surkhet' 'Bharatpur' 'Bhairahawa' 'Biratnagar' 'Janakpur'
 'Taplejung' 'POKHARA' 'NEPALGUNJ' nan 'Ramechhap' 'Varanasi' 'Simara'
 'JANAKPUR' 'SIMARA' 'BIRATNAGAR' 'DHANGADHI' 'BHAIRAHAWA' 'MTN' 'Dang'
 'Rajbiraj' 'Mountain Flight' 'BHARATPUR' 'BHADRAPUR' 'Resunga'
 'TUMLINGTAR' 'Tumlingtar' 'RAMECHHAP' 'Tikapur' 'RAJBIRAJ' 'Mountain'
 'LUKLA' 'Bhojpur' 'BAJURA' 'Kolkata' 'JUMLA' 'SURKHET' 'Jumla' 'DANG'
 'Ilam' 'Rumjatar' 'JOMSOM' 'Bajhang' 'TAPLEJUNG' 'Jomsom' 'BHOJPUR'
 'Lukla' 'Rara' 'DOLPA' 'Rukum' 'Bajura' 'Simikot' 'BAJHANG' 'PHAPLU'
 'Khanidanda' 'VARANASI' 'Phaplu' 'TIKAPUR' 'Dolpa' 'SIMIKOT' 'RARA'
 'TALCHA' 'KHANIDANDA' 'RUKUM' 'Kamalbazar' 'KAMALBAZAR' 'MOUNTAIN'
 'Thamkharka' 'SANFEBAGAR' 'Sanfebagar' 'THAMKHARKA']


In [13]:
#convert destionation column to proper case in Hotel1 DataFrame
#filter hotel 1 where event_date is between 2025-06-26 and 2025-07-26
#Hotel1 = Hotel1[(Hotel1['event_date'] >= '2025-06-26') & (Hotel1['event_date'] <= '2025-07-26')]
#Hotel1['destination'] = Hotel1['destination'].str.title()
Domestic_Airlines1 = Domestic_Airlines1[(Domestic_Airlines1['event_date'] >= '2025-10-15') & (Domestic_Airlines1['event_date'] <= '2025-11-01')]
Domestic_Airlines1['destinationCity'] = Domestic_Airlines1['destinationCity'].str.title()
#unique count of user_id based on destination for Hotel1 DataFrame
unique_user_count = Domestic_Airlines1.groupby('destinationCity')['user_id'].nunique().reset_index()
unique_user_count.columns = ['destinationCity', 'unique_event_count']
# Sort the unique user count DataFrame by unique_user_count in descending order
unique_user_count = unique_user_count.sort_values(by='unique_event_count', ascending=False)
#display result
display(unique_user_count.head(10))
# Save the unique user count DataFrame to a CSV file with name 'unique_user_count_during-event.csv'
#unique_user_count.to_csv(r"D:\Hotel_Experiment\Parquet\unique_user_count_during-event.csv", index=False)


,destinationCity,unique_event_count
15,Kathmandu,154515
6,Biratnagar,29110
24,Pokhara,27151
22,Nepalgunj,23752
3,Bhairahawa,22942
8,Dhangadhi,20806
11,Janakpur,18034
2,Bhadrapur,13757
4,Bharatpur,13580
32,Simara,12623


In [14]:
#Filter Domestic_Airlines1 DataFrame to get rows where destinationCity is either POKHARA or Pokhara or Lumbini or LUMBINI
Domestic_Airlines1 = Domestic_Airlines1[Domestic_Airlines1['destinationCity'].str.contains('POKHARA|Pokhara|BHAIRAHAWA|Bhairahawa', case=False, na=False)]
#Filter Hotel1 DataFrame to get rows where destination is either POKHARA or Pokhara or Lumbini or LUMBINI
#Hotel1 = Hotel1[Hotel1['destination'].str.contains('POKHARA|Pokhara|LUMBINI|Lumbini', case=False, na=False)]
#Display the first 5 rows of the filtered Domestic_Airlines1 DataFrame
display(Domestic_Airlines1.head(5))
#Display the shape of the filtered Domestic_Airlines1 DataFrame
print("Filtered Domestic Airlines DataFrame shape:", Domestic_Airlines1.shape)
#print distinct account id from Domestic_Airlines1 DataFrame
print("Distinct account id from Domestic Airlines DataFrame:")
print(Domestic_Airlines1['user_id'].nunique())
#Groupby Domestic airlines 1 event_name and get the distinct count of each user_id
grouped = Domestic_Airlines1.groupby('event_name')['user_id'].nunique().reset_index()
#Rename the columns of the grouped DataFrame
grouped.columns = ['event_name', 'distinct_user_count']
#Sort the grouped DataFrame by distinct_user_count in descending order
grouped = grouped.sort_values(by='distinct_user_count', ascending=False)

#Display the grouped DataFrame
display(grouped.head(5))

#Group by Hotel1 event_name and get the distinct count of each user_id
#grouped_hotel = Hotel1.groupby('event_name')['user_id'].nunique().reset_index()
#Rename the columns of the grouped_hotel DataFrame
#grouped_hotel.columns = ['event_name', 'distinct_user_count']
#Sort the grouped_hotel DataFrame by distinct_user_count in descending order
#grouped_hotel = grouped_hotel.sort_values(by='distinct_user_count', ascending=False)
#Display the grouped_hotel DataFrame
#display(grouped_hotel.head(5))
#Domestic_Airlines1.to_csv(r"C:\Users\lenovo\Downloads\April_03-05_DA_Pokhara_Bhairahawa_Data.csv", index=False)
#Hotel1.to_csv(r"C:\Users\lenovo\Downloads\April_03-05_Hotel_Pokhara_Lumbini_Data.csv", index=False)
#Display the shape of the filtered Hotel1 DataFrame
#print("Filtered Hotel DataFrame shape:", Hotel1.shape)


,user_id,event_name,category,unique_category,event_date,event_timestamp,confirmed,departureCity,adults,tripType,...,time,departureDate,user_type,accountHolderName,esewaAccountId,medium,source,platform,version,install_source
48347,698,domesticFlightDetail,Domestic Airlines,Domestic Airlines,2025-11-01,2025-11-01 04:54:41,NaN,Kathmandu,1,One Way,...,09:07:26,2025-10-24,NaN,NaN,NaN,(none),(direct),ANDROID,4.7.0.1,com.android.vending
342058,1146,domesticFlightDetail,Domestic Airlines,Domestic Airlines,2025-10-26,2025-10-26 12:54:04,NaN,Kathmandu,1,One Way,...,19:15:30,2025-10-25,NaN,NaN,NaN,(none),(direct),ANDROID,4.7.0.1,com.android.vending
391788,1322,domesticFlightDetail,Domestic Airlines,Domestic Airlines,2025-10-15,2025-10-15 07:54:48,NaN,Kathmandu,1,One Way,...,NaN,NaN,NaN,NaN,NaN,organic,google-play,ANDROID,4.7.0.1,com.android.vending
408077,1586,domesticFlightSearch,Domestic Airlines,Domestic Airlines,2025-11-01,2025-11-01 10:32:08,NaN,Kathmandu,1,One Way,...,13:40:14,2025-10-24,NaN,NaN,NaN,(none),(direct),IOS,5.5.27,iTunes
453783,1720,domesticFlightSearch,Domestic Airlines,Domestic Airlines,2025-10-23,2025-10-23 10:04:12,NaN,Kathmandu,1,Two Way,...,18:13:50,2025-10-20,NaN,NaN,NaN,(none),(direct),IOS,5.5.27,iTunes


Filtered Domestic Airlines DataFrame shape: (57449, 23)
Distinct account id from Domestic Airlines DataFrame:
45689


,event_name,distinct_user_count
1,domesticFlightDetail,34350
3,domesticFlightSearch,11622
0,domesticFlightConfirmation,872
2,domesticFlightPassengerDetail,6


In [ ]:
funnel_data = {
    "Stage": ["domesticFlightSearch", "domesticFlightDetail", "domesticFlightPassengerDetail", "domesticFlightConfirmation"],
    "Count": [
        len(df[df['event_name'] == 'domesticFlightSearch'])+len(df[df['event_name'] == 'domesticFlightDetail'])+len(df[df['event_name'] == 'domesticFlightConfirmation'])+len(df[df['event_name'] == 'domesticFlightPassengerDetail']),
        len(df[df['event_name'] == 'domesticFlightDetail'])+len(df[df['event_name'] == 'domesticFlightConfirmation'])+len(df[df['event_name'] == 'domesticFlightPassengerDetail']),
        len(df[df['event_name'] == 'domesticFlightPassengerDetail']) +len(df[df['event_name'] == 'domesticFlightConfirmation']),
        len(df[df['event_name'] == 'domesticFlightConfirmation']) ]}

In [ ]:
import plotly.graph_objects as go

funnel_df = pd.DataFrame(funnel_data)

# Calculate drop-off percentages
dropoff_percentages = [0] # Drop-off from a hypothetical stage before the first is 0
for i in range(1, len(funnel_df['Count'])):
    dropoff = funnel_df['Count'][i-1] - funnel_df['Count'][i]
    dropoff_percent = (dropoff / funnel_df['Count'][i-1]) * 100 if funnel_df['Count'][i-1] > 0 else 0
    dropoff_percentages.append(dropoff_percent)

# Create a list of text labels, including drop-off percentages
text_labels = []
for i, stage in enumerate(funnel_df['Stage']):
    conversion_rate = (funnel_df['Count'][i]/funnel_df['Count'][0]) * 100 if funnel_df['Count'][0] > 0 else 0
    if i == 0:
        text_labels.append(f"{funnel_df['Count'][i]} ({conversion_rate:.1f}%)")
    else:
         text_labels.append(f"{funnel_df['Count'][i]} ({conversion_rate:.1f}%) - {dropoff_percentages[i]:.1f}% Drop-off")


# Plot funnel chart using Plotly
fig = go.Figure(go.Funnel(
    y=funnel_df['Stage'],  # Y-axis represents the stages of the funnel
    x=funnel_df['Count'],  # X-axis represents the number of users at each stage
    textinfo="text",  # Show the custom text labels
    text=text_labels, # Assign the custom text labels
    textposition="auto" # Position the text outside the bars
))

# Update layout with a title and size
fig.update_layout(
    title="User Activation & Purchase Funnel with Drop-off Analysis",
    autosize=False, # Disable autosize
    width=1000, # Set the width of the chart
    height=900 # Set the height of the chart
)

# Show the interactive funnel chart
fig.show()